In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import torch
from qgsw.spatial.core.grid_conversion import cell_corners_to_cell_center
import matplotlib.pyplot as plt
from pathlib import Path
import os
from qgsw.utils.sorting import sort_files
from qgsw.run_summary import RunSummary
ROOT = Path(os.path.abspath('')).parent

In [ ]:
H1 = 200
H2 = 800
g1 = 10
g2 = 0.05


A = np.array([
    [1/H1/g1 + 1/H1/g2, -1/H1/g2],
    [-1/H2/g2, 1/H2/g2]
])
eigvals, P = np.linalg.eig(A)
D = torch.tensor(np.diag(eigvals), dtype=torch.float64)
P = torch.tensor(P, dtype=torch.float64)

In [ ]:
def abs_thresh(data:torch.Tensor, threshold:float = 1e-5)-> torch.Tensor:
    return torch.abs(data) > threshold

def rel_thresh(data: torch.Tensor, threshold:float = 0.01) -> torch.Tensor:
    return torch.abs(data) > threshold * torch.mean(torch.abs(data))


In [ ]:
%matplotlib tk

baroclinic_30 = "two_layers_baroclinic_30km"
barotropic_100 = "two_layers_barotropic_100km"
baroclinic_100 = "two_layers_baroclinic_100km"

path = ROOT.joinpath(f"output/g5k/{baroclinic_30}")
run = RunSummary.from_file(path.joinpath("_summary.toml"))
steps, files_baroclinic_30 = sort_files(list(path.glob(f"{run.configuration.model.prefix}*.npz")),run.configuration.model.prefix,".npz")

path = ROOT.joinpath(f"output/g5k/{barotropic_100}")
run = RunSummary.from_file(path.joinpath("_summary.toml"))
steps, files_barotropic_100 = sort_files(list(path.glob(f"{run.configuration.model.prefix}*.npz")),run.configuration.model.prefix,".npz")

path = ROOT.joinpath(f"output/g5k/{baroclinic_100}")
run = RunSummary.from_file(path.joinpath("_summary.toml"))
steps, files_baroclinic_100 = sort_files(list(path.glob(f"{run.configuration.model.prefix}*.npz")),run.configuration.model.prefix,".npz")

plt.ion()


plt.figure(figsize=(12, 12))
ax1 = plt.subplot(2,3,1)
ax2 = plt.subplot(2,3,2)
ax3 = plt.subplot(2,3,3)
ax4 = plt.subplot(2,1,2)
axes = [ax1, ax2, ax3, ax4]

offset = 24

modes_1_baroclinic_30 = [np.nan for _ in files_baroclinic_30]
modes_2_baroclinic_30 = [np.nan for _ in files_baroclinic_30]
modes_1_barotropic_100 = [np.nan for _ in files_barotropic_100]
modes_2_barotropic_100 = [np.nan for _ in files_barotropic_100]
modes_1_baroclinic_100 = [np.nan for _ in files_baroclinic_100]
modes_2_baroclinic_100 = [np.nan for _ in files_baroclinic_100]

times = [step * run.configuration.simulation.dt for step in steps]

for i in range(len(times)):

    axes[0].cla()
    axes[1].cla()
    axes[2].cla()
    axes[3].cla()

    psi = torch.tensor(np.load(files_baroclinic_30[i])["p"], dtype=torch.float64)[...,offset:-offset,offset:-offset]
    modes = torch.einsum("lm,...mxy->...lxy", torch.inverse(P), psi)
    top = modes[0,0,...]
    bottom = modes[0,1,...]


    threshold_psi_top = abs_thresh(psi[0,0,...], 1e-2)
    axes[0].imshow(threshold_psi_top)

    mode_1 = torch.mean(torch.abs(top[threshold_psi_top]))
    mode_2 = torch.mean(torch.abs(bottom[threshold_psi_top]))

    modes_1_baroclinic_30[i] = mode_1 / (mode_1 + mode_2)
    modes_2_baroclinic_30[i] = mode_2 / (mode_1 + mode_2)

    ax4.plot(times,modes_1_baroclinic_30, label = "1st Baroclinic Mode - Baroclinic 30 km", marker='o', c='r')
    ax4.plot(times,modes_2_baroclinic_30, label = "Barotropic Mode - Baroclinic 30 km", marker='o', c='g')

    psi = torch.tensor(np.load(files_barotropic_100[i])["p"], dtype=torch.float64)[...,offset:-offset,offset:-offset]
    modes = torch.einsum("lm,...mxy->...lxy", torch.inverse(P), psi)
    top = modes[0,0,...]
    bottom = modes[0,1,...]


    threshold_psi_top = abs_thresh(psi[0,0,...], 1e-2)
    axes[1].imshow(threshold_psi_top)

    mode_1 = torch.mean(torch.abs(top[threshold_psi_top]))
    mode_2 = torch.mean(torch.abs(bottom[threshold_psi_top]))

    modes_1_barotropic_100[i] = mode_1 / (mode_1 + mode_2)
    modes_2_barotropic_100[i] = mode_2 / (mode_1 + mode_2)

    ax4.plot(times,modes_1_barotropic_100, label = "1st Baroclinic Mode - Barotropic 100 km", marker='x', c='r')
    ax4.plot(times,modes_2_barotropic_100, label = "Barotropic Mode - Barotropic 100 km", marker='x', c='g')


    psi = torch.tensor(np.load(files_baroclinic_100[i])["p"], dtype=torch.float64)[...,offset:-offset,offset:-offset]
    modes = torch.einsum("lm,...mxy->...lxy", torch.inverse(P), psi)
    top = modes[0,0,...]
    bottom = modes[0,1,...]


    threshold_psi_top = abs_thresh(psi[0,0,...], 1e-2)
    axes[2].imshow(threshold_psi_top)

    mode_1 = torch.mean(torch.sqrt(torch.square(top[threshold_psi_top])))
    mode_2 = torch.mean(torch.sqrt(torch.square(bottom[threshold_psi_top])))

    modes_1_baroclinic_100[i] = mode_1 / (mode_1 + mode_2)
    modes_2_baroclinic_100[i] = mode_2 / (mode_1 + mode_2)

    ax4.plot(times,modes_1_baroclinic_100, label = "1st Baroclinic Mode - Baroclinic 100 km", marker='s', c='r')
    ax4.plot(times,modes_2_baroclinic_100, label = "Barotropic Mode - Baroclinic 100 km", marker='s', c='g')

    ax4.set_xlim(times[0], times[-1])
    ax4.set_ylim(0,1)
    ax4.legend()

    plt.pause(0.01)
plt.ioff()


In [ ]:
file

In [ ]:
psi[0,0,...]